In [50]:
import numpy as np
import h5py
import json
import pandas as pd


In [51]:
# Read the CSV file
training_data_file = 'C:/Users/myrsi/Documents/msc_thesis/training_data.csv'
validation_data_file = 'C:/Users/myrsi/Documents/msc_thesis/validation_data.csv'

df_training_data = pd.read_csv(training_data_file)
df_validation_data = pd.read_csv(validation_data_file)


In [52]:
# Access the data in the DataFrame
# For example, print the first 5 rows of the DataFrame
print(df_training_data.head())
print(df_validation_data.head())

      Angle        RCS  Range_Sc  Doppler_Vel  Label_ID
0 -0.124031 -28.300673  2.566963     0.009330        11
1  0.849328 -26.863953  2.096815    -8.023678        11
2  1.106456 -26.564907  3.338323    -9.334642        11
3  1.156905   0.101503  8.882484    -7.361293        11
4  1.090472  -3.724234  9.479269    -7.834501        11
      Angle        RCS   Range_Sc  Doppler_Vel  Label_ID
0 -0.949410  -8.967562  15.204274     0.013444        11
1 -0.927236  -8.756395   8.291416    -0.204751        11
2 -0.318540 -23.922726  10.212634    -0.002552        11
3  0.490461 -20.036438  10.412143     0.002362        11
4  0.385698 -11.150873  16.565825    -0.001025        11


In [53]:
# Check for NaN values
has_nan = df_training_data.isna().any().any()
#has_nan = df_validation_data.isna().any().any()

# Print the result
if has_nan:
    print("There are NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")

There are no NaN values in the DataFrame.


In [54]:
# Check if 'Doppler_Vel' column has zero values
#has_zeros = any(df_training_data['Doppler_Vel'] == 0)
has_zeros = any(df_validation_data['Doppler_Vel'] == 0)

if has_zeros:
    print("The 'Doppler_Vel' column contains zero values.")
else:
    print("The 'Doppler_Vel' column does not contain zero values.")



The 'Doppler_Vel' column contains zero values.


In [55]:
# Remove rows with non-zero values in 'Doppler_Vel' column

df_training_data_static = df_training_data[df_training_data['Doppler_Vel'] == 0]
df_validation_data_static = df_validation_data[df_validation_data['Doppler_Vel'] == 0]


In [56]:
print(df_training_data_static.head())
print(df_validation_data_static.head())

           Angle        RCS   Range_Sc  Doppler_Vel  Label_ID
8466    0.073331 -15.673953  36.329308          0.0        11
92228   0.087288   7.818957  37.510777          0.0        11
322656  1.112901   8.035492  10.215149          0.0        11
335977  0.101549 -12.426669  11.838837          0.0        11
394798  0.661913 -25.315552   5.563264          0.0        11
         Angle        RCS   Range_Sc  Doppler_Vel  Label_ID
450   0.554729 -22.479916  10.081914          0.0        11
482  -0.291218  21.703321  51.351340          0.0        11
679   0.102620  26.898014  68.092430          0.0        11
1180 -0.314791  15.404928  48.002570          0.0        11
1189 -0.244994  19.542840  57.628696          0.0        11


In [57]:
# Check if 'Doppler_Vel' column has any non-zero values
has_non_zeros = any(df_training_data_static['Doppler_Vel'] != 0)
#has_non_zeros = any(df_validation_data_static['Doppler_Vel'] != 0)

if has_non_zeros:
    print("The 'Doppler_Vel' column contains non-zero values.")
else:
    print("The 'Doppler_Vel' column does not contain any non-zero values.")


The 'Doppler_Vel' column does not contain any non-zero values.


In [ ]:
# Set the features and labels. Remove the extra column(Doppler_vel) because i don't need it anymore.

In [58]:
X = df_training_data_static[['Angle','RCS','Range_Sc']]
y = df_training_data_static['Label_ID']
X_val = df_validation_data_static[['Angle','RCS','Range_Sc',]]
y_val = df_validation_data_static['Label_ID']

In [59]:
print(X.head())
print(y.head())
print(X_val.head())
print(y_val.head())

           Angle        RCS   Range_Sc
8466    0.073331 -15.673953  36.329308
92228   0.087288   7.818957  37.510777
322656  1.112901   8.035492  10.215149
335977  0.101549 -12.426669  11.838837
394798  0.661913 -25.315552   5.563264
8466      11
92228     11
322656    11
335977    11
394798    11
Name: Label_ID, dtype: int64
         Angle        RCS   Range_Sc
450   0.554729 -22.479916  10.081914
482  -0.291218  21.703321  51.351340
679   0.102620  26.898014  68.092430
1180 -0.314791  15.404928  48.002570
1189 -0.244994  19.542840  57.628696
450     11
482     11
679     11
1180    11
1189    11
Name: Label_ID, dtype: int64


In [60]:
import torch
torch.__version__

'1.12.1'

In [61]:
type(X), type(y), type(X_val), type(y_val)

(pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series)

In [63]:
# Turn data into tensors
# Otherwise this causes issues with computations later on
X = torch.Tensor(df_training_data_static[['Angle', 'RCS', 'Range_Sc']].values)
y = torch.Tensor(df_training_data_static['Label_ID'].values)

X_val = torch.Tensor(df_validation_data_static[['Angle', 'RCS', 'Range_Sc']].values)
y_val = torch.Tensor(df_validation_data_static['Label_ID'].values)


In [68]:
# View the first five samples
X[:5], y[:5], X_val[:5], y_val[:5]
X.dtype, y.dtype, X_val.dtype, y_val.dtype

(torch.float32, torch.float32, torch.float32, torch.float32)

In [69]:
len(X), len(y), len(X_val), len(y_val)

(143743, 143743, 38508, 38508)

In [71]:
from sklearn.model_selection import train_test_split
val_proportion = df_validation_data_static.shape[0] / df_training_data_static.shape[0]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=val_proportion,random_state=42)

In [72]:
len(X_train), len(X_test), len(X_val), len(y_train), len(y_test), len(y_val)

(105235, 38508, 38508, 105235, 38508, 38508)

In [74]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
val_dataset = TensorDataset(X_val, y_val)

# Create DataLoaders
batch_size = 5
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [75]:
import torch
from torch import nn

# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
# chat gpt updated version
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Read the CSV file
training_data_file = 'C:/Users/myrsi/Documents/msc_thesis/training_data.csv'
validation_data_file = 'C:/Users/myrsi/Documents/msc_thesis/validation_data.csv'
df_training_data = pd.read_csv(training_data_file)
df_validation_data = pd.read_csv(validation_data_file)

# Remove rows with non-zero values in 'Doppler_Vel' column
df_training_data_static = df_training_data[df_training_data['Doppler_Vel'] == 0]
df_validation_data_static = df_validation_data[df_validation_data['Doppler_Vel'] == 0]

# Extract features and labels
X_train = df_training_data_static[['Angle', 'RCS', 'Range_Sc']].values
y_train = df_training_data_static['Label_ID'].values
X_val = df_validation_data_static[['Angle', 'RCS', 'Range_Sc']].values
y_val = df_validation_data_static['Label_ID'].values

# Split the training data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=val_proportion, random_state=42)

# Convert data to tensors
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)
X_val = torch.Tensor(X_val)
y_val = torch.Tensor(y_val)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
val_dataset = TensorDataset(X_val, y_val)

# Create DataLoaders
batch_size = 5
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [88]:
import torch.nn as nn
import torch.nn.functional as F

# Define the model architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=6)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=9)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.upsample1 = nn.Upsample(size=2)
        self.deconv1 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=9)
        self.bn5 = nn.BatchNorm2d(128)
        self.deconv2 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=6)
        self.bn6 = nn.BatchNorm2d(64)
        self.upsample2 = nn.Upsample(size=2)
        self.deconv3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=5)
        self.bn7 = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(in_channels=32, out_channels=6, kernel_size=3)

        
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))) 
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)    
        x = F.relu(self.bn3(self.conv3(x))) 
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)     

        x = self.upsample1(x)
        x = F.relu(self.bn5(self.deconv1(x))) 
        x = F.relu(self.bn6(self.deconv2(x)))
        x = self.upsample2(x)
        x = F.relu(self.bn7(self.deconv3(x))) 
        x = self.deconv4(x)


        x = x.view(-1, 64 * 8 * 8)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.softmax(self.fc2(x))

        return x

##for fc linear im not sure. not in article.   

#instantiate a neural network model

model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(6, 6), stride=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(9, 9), stride=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (upsample1): Upsample(size=2, mode=nearest)
  (deconv1): ConvTranspose2d(256, 128, kernel_size=(9, 9), stride=(1, 1))
  (bn5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv

In [124]:
X_train.shape
X_train

BATCH_SIZE = 5
print(f"Dataloaders: {train_dataloader, test_dataloader, val_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of val dataloader: {len(val_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x00000217AEE62320>, <torch.utils.data.dataloader.DataLoader object at 0x00000215A05C3AC0>, <torch.utils.data.dataloader.DataLoader object at 0x00000217AEE62BC0>)
Length of train dataloader: 21047 batches of 5
Length of test dataloader: 7702 batches of 5
Length of val dataloader: 7702 batches of 5


In [125]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([5, 3]), torch.Size([5]))

In [99]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=6)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=9)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.upsample1 = nn.Upsample(size=2)
        self.deconv1 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=9)
        self.bn5 = nn.BatchNorm2d(128)
        self.deconv2 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=6)
        self.bn6 = nn.BatchNorm2d(64)
        self.upsample2 = nn.Upsample(size=2)
        self.deconv3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=5)
        self.bn7 = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(in_channels=32, out_channels=6, kernel_size=3)

        self.fc1 = nn.Linear(64 * 8 * 8, 256)  # Add a fully connected layer
        self.fc2 = nn.Linear(256, 6)  # Add another fully connected layer
        self.dropout = nn.Dropout(p=0.5)  # Add a dropout layer

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)

        x = self.upsample1(x)
        x = F.relu(self.bn5(self.deconv1(x)))
        x = F.relu(self.bn6(self.deconv2(x)))
        x = self.upsample2(x)
        x = F.relu(self.bn7(self.deconv3(x)))
        x = self.deconv4(x)

        x = x.view(-1, 32 * 8 * 8)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.softmax(self.fc2(x))

        return x
    
    
model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(6, 6), stride=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(9, 9), stride=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (upsample1): Upsample(size=2, mode=nearest)
  (deconv1): ConvTranspose2d(256, 128, kernel_size=(9, 9), stride=(1, 1))
  (bn5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv

In [117]:
# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9,0.999))

In [118]:
torch.manual_seed(42)

epochs = 5

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)
X_val, y_val = X_val.to(device), y_val.to(device)


for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        X_train, y_train = data

        # forward pass
        y_pred = model(X_train).squeeze()
        
        # Calculate loss
        loss = loss_fn(y_pred, y_train)
        
        # Optimizer zero grad
        optimizer.zero_grad()
        
        # Loss backward 
        loss.backward
        
        # Optimizer step
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: expected scalar type Long but found Float